In [1]:
%load_ext autoreload
%autoreload 1
import sys
import os
from pathlib import Path
sys.path.insert(1, str(Path(os.getcwd()).resolve().parent))

import pandas as pd 
pd.set_option('display.max_columns', 150)

from project.utils.data import *
%aimport project.utils.data

In [2]:
df = load_extended_data()

In [3]:
### to do

# choose df columns s. t. feature selection results 

In [4]:
df = df.drop(columns=['January', 'February', 'March',
       'April', 'May', 'June', 'July', 'August', 'September', 'October',
       'November', 'December',])

In [5]:
# creating lag features
X = (
    project.utils.data.create_lag_features(df, 'balance', [1, 7, 14])
    .drop(columns=['balance', 'income', 'outcome'])
)

# cutting nulls after creating lags 
start = X[X.isnull().sum(axis=1) == 0].index.min()

y, X = df['balance'][start:], X[start:]

assert y.shape[0] == X.shape[0], 'alarm (shape)'
assert y.index.min() == X.index.min(), 'alarm (dates)'

In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error

def time_series_model_fit(target,
                          features,
                          params_grid,
                          model_class,
                          cv_window='expanding',
                          n_splits=5,
                          additional_metric=None,
                          test_size=0.2
                         ):
    # Step 1: Split time series data into train and test
    split_index = int(len(target) * (1 - test_size))
    target_train, target_test = target[:split_index], target[split_index:]
    features_train, features_test = features[:split_index], features[split_index:]

    # Step 2: Define Time Series Cross-Validation
    if cv_window == 'expanding':
        cv = TimeSeriesSplit(n_splits=n_splits if n_splits else len(target_train))
    elif cv_window == 'rolling':
        cv = TimeSeriesSplit(n_splits=n_splits)

    # Step 3: Run Grid Search CV on the train data
    grid_search = GridSearchCV(model_class, params_grid, cv=cv, scoring='neg_mean_absolute_error')
    grid_search.fit(features_train, target_train)

    best_model = grid_search.best_estimator_

    # Step 4: Evaluate the model on the test data
    predictions_test = best_model.predict(features_test)
    mae_test = mean_absolute_error(target_test, predictions_test)

    # Step 5: Fit the best model on all data
    best_model.fit(features, target)

    additional_metric_result = None
    if additional_metric:
        additional_metric_result = additional_metric(target_test, predictions_test)

    return best_model, mae_test, additional_metric_result, grid_search.best_params_



In [30]:
from catboost import  CatBoostRegressor

In [35]:
param_grid = {  
    'iterations': [100, 200, 300],  
    'learning_rate': [0.1, 1],  
    'depth': [5, 7, 8],  
    # 'l2_leaf_reg': [1, 5, 10],  
    # 'random_strength': [1, 5, 10],  
    # 'bagging_temperature': [0.5, 1, 2],  
    # 'random_seed': [0, 42, 123]  
}  
  
model = CatBoostRegressor(verbose=0) 

In [14]:
# y.to_numpy()

In [36]:
result = time_series_model_fit(
    target=y,
    features=X, params_grid=param_grid, model_class=model, cv_window='rolling'
)

In [37]:
result

(<catboost.core.CatBoostRegressor at 0x2b4707e80>,
 0.01528840902027627,
 None,
 {'depth': 5, 'iterations': 300, 'learning_rate': 0.1})